In [5]:
# made by self
from lib import gridmap as gm
from lib import makepath as mp
from lib import cartrobot as cb

import numpy as np
import math

OBS_W = 0.9
VEL_W = 0.3
ANG_W = 0.7

def normalize(data):
    data = np.array(data)
    max_data = max(data)
    min_data = min(data)

    if max_data == min_data:
        data = np.array([0.0 for i in range(len(data))])
    else:
        data = (data - min_data) / (max_data - min_data)

    return data

def correction_ang(angle):
    if angle > math.pi:
        while angle > math.pi:
            angle -= 2*math.pi
    elif angle < -math.pi:
        while angle < -math.pi:
            angle += 2*math.pi

    return np.array(angle)

class Evaluator():
    def __init__(self):
        self.ang_scores = []
        self.obs_scores = []
        self.vel_scores = []
    
    def calc_input(self, paths, goal, obstacles):
        grid_map, x_min, y_min, grid_size = gm.main(paths, obstacles)

        for path in paths:
            self.calc_obs_score(path, grid_map, x_min, y_min, grid_size)
            self.calc_vel_score(path[0][3])
            self.calc_ang_score(path[-1], goal[0], goal[1])

        normed_obs_score = normalize(self.obs_scores)
        normed_vel_score = normalize(self.vel_scores)
        normed_ang_score = normalize(self.ang_scores)
        
        total_score = OBS_W * normed_obs_score + VEL_W * normed_vel_score + ANG_W * normed_ang_score
        best_score_id = np.argmax(total_score)
        evaluated_input = paths[best_score_id][-1][3:6]

        return evaluated_input, grid_map #本使用ではgrid_map消す      

    def calc_obs_score(self, path, grid_map, x_min, y_min, grid_size):
        score = 0.0
        for (x, y, th, v, o) in path:
            x_i = int(round((x-x_min)/grid_size))
            y_i = int(round((y-y_min)/grid_size))
            if grid_map[x_i][y_i] > score:
                score = grid_map[x_i][y_i]
        self.obs_scores.append(1-score)

    def calc_vel_score(self, v):
        self.vel_scores.append(v)
    
    def calc_ang_score(self, last_status, g_x, g_y):
        self.ang_scores.append(
            math.pi-abs(
                correction_ang(
                    math.atan2(g_y-last_status[1], g_x-last_status[1]) - last_status[2]
                )
            )
        )

if __name__ == "__main__":
    import csv
    import time
    start = time.time()
    cartbot = cb.main(0, 0, math.pi/2)
    #LRFからのデータに変える
    np.random.seed(1)
    obstacles = [list((np.random.rand(2)-0.5)*10) for i in range(30)]
    goal = [10, 10]

    for _ in range(100):
        evals = Evaluator()
        paths = mp.main(cartbot.status)

        inputs, grid_map = evals.calc_input(paths, goal, obstacles)
        cartbot.update_status(inputs)
        print(cartbot.status)

    finished = time.time()-start
    print(finished)
    
    with open("gridmap.csv", "w") as file:
        write = csv.writer(file, lineterminator="\n")
        write.writerows(grid_map)

[0.0, 0.0, 1.4660765716752369, 0.0, -0.20943951023931953]
[0.0, 0.0, 1.2915436464758039, 0.0, -0.3490658503988658]
[0.08269120674509976, 0.28837850878149573, 1.1606439525762293, 0.6000000000000001, -0.2617993877991489]
[0.20231592742267362, 0.563496531097033, 1.0646508437165416, 0.6000000000000001, -0.1919862177193753]
[0.5174421805827923, 1.1319993407376405, 0.9948376736367691, 1.3, -0.139626340159545]
[0.8714575533425593, 1.6771352099021666, 0.9424777960769402, 1.2999999999999998, -0.10471975511965803]
[1.2535179673326655, 2.202996256245883, 0.898844564777083, 1.2999999999999996, -0.08726646259971438]
[1.6892782129789974, 2.750821966042574, 0.8726646259971693, 1.3999999999999995, -0.052359877559827406]
[2.1070901592752453, 3.2487508540699115, 0.8464846872172556, 1.2999999999999994, -0.052359877559827406]
[2.5709241930262583, 3.7730198586222152, 0.8290313946973135, 1.399999999999999, -0.034906585039884114]
[3.0438373383572164, 4.289113994389305, 0.820304748437343, 1.3999999999999988, 

[8.539912034889165, 9.893164613465977, 0.7679448708775665, 0.0, 1.6653345369377348e-16]
[8.539912034889165, 9.893164613465977, 0.7679448708775666, 0.0, 1.3877787807814457e-16]
[8.539912034889165, 9.893164613465977, 0.7679448708775667, 0.0, 1.1102230246251565e-16]
[8.539912034889165, 9.893164613465977, 0.7679448708775667, 0.0, 8.326672684688674e-17]
[8.539912034889165, 9.893164613465977, 0.7679448708775667, 0.0, 5.551115123125783e-17]
[8.539912034889165, 9.893164613465977, 0.7679448708775667, 0.0, 2.7755575615628914e-17]
[8.539912034889165, 9.893164613465977, 0.7679448708775667, 0.0, 0.0]
[8.539912034889165, 9.893164613465977, 0.7679448708775667, 0.0, -2.7755575615628914e-17]
15.037882804870605


0.0


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


[0.8, -0.20943951023931953]

In [33]:
b

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])